## Regular daily execution of repair volume

In [1]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
from datetime import datetime, timedelta
from sklearn import preprocessing
import os
from matplotlib import pyplot as plt
from multiprocessing import Pool

In [2]:
from pandas.tseries.holiday import USFederalHolidayCalendar, AbstractHolidayCalendar, Holiday, \
    nearest_workday, USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
    USLaborDay, USThanksgivingDay

rules = {
            'NewYearsDay':Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
            'USMartinLutherKingJr':USMartinLutherKingJr,
            'USPresidentsDay':USPresidentsDay,
            'GoodFriday':GoodFriday,
            'USMemorialDay':USMemorialDay,
            'Holiday':Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
            'USLaborDay':USLaborDay,
            'USThanksgivingDay':USThanksgivingDay,
            'Holiday':Holiday('Christmas', month=12, day=25, observance=nearest_workday)
        }

holidays_list = {}
for key, rule in rules.items():
    holidays_list[key] = AbstractHolidayCalendar(rules = [ rule ]).holidays(datetime(2018-1, 12, 31), datetime(2021, 12, 31))

def get_holiday_name(date):
    for key, rule in holidays_list.items():
        if date in holidays_list[key]:
            return key
    return "No_holiday"

#holidays_list

In [3]:
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2018-01-01', end='2021-12-31').to_pydatetime()

### Evaluation function

In [4]:
# self-defined eval metric
# f(y_true: array, y_pred: array) -> name: string, eval_result: float, is_higher_better: bool
# SMAPE
def smape_eval(A, F):
    return 'SMAPE', smape(A, F), False

def smape(A, F):
    return round(np.sum(np.abs(F - A)) / np.sum(A + F), 4)

### Feature engineering

In [5]:
# frame a sequence as a supervised learning problem
#def timeseries_to_supervised(data, lags=89, custom_lags=[90], windows=[7, 8, 13, 21, 34, 55]):
def timeseries_to_supervised(data, lags=8, custom_lags=[14, 21, 28, 35, 42, 49, 56, 63, 70], windows=[7, 8, 13, 21, 34]):    
#def timeseries_to_supervised(data, lags=30, custom_lags=[35, 42, 49, 56, 63, 70], windows=[7, 8, 13, 14, 21, 28, 34, 35, 42, 49, 55]):
    #print(data.head())
    data['month'] = data.date.dt.month
    data['year'] = data.date.dt.year
    data['week'] = data.date.dt.isocalendar().week
    data['weekday'] = data.date.dt.dayofweek
    data['monthday'] = data.date.dt.day   
    data['holiday'] = data['date'].isin(holidays)*1
    data['holiday_name'] = data.date.apply(get_holiday_name)
    le_hol = preprocessing.LabelEncoder()
    data['holiday_name'] = le_hol.fit_transform(data.holiday_name)

    for i in windows:
        #print(i)
        data['rolling_mean_'+str(i)] = data['volume'].shift(1).rolling(i).mean().fillna(0)
        data['rolling_median_'+str(i)] = data['volume'].shift(1).rolling(i).median().fillna(0)
        data['rolling_min_'+str(i)] = data['volume'].shift(1).rolling(i).min().fillna(0)
        data['rolling_max_'+str(i)] = data['volume'].shift(1).rolling(i).max().fillna(0)
        data['rolling_std_'+str(i)] = data['volume'].shift(1).rolling(i).std().fillna(0)
        data['rolling_kurt_'+str(i)] = data['volume'].shift(1).rolling(i).kurt().fillna(0)
        data['rolling_skew_'+str(i)] = data['volume'].shift(1).rolling(i).skew().fillna(0)
        data['rolling_var_'+str(i)] = data['volume'].shift(1).rolling(i).var().fillna(0)
        
    for i in range(1, lags+1):
        data['y_lag_'+str(i)] = data['volume'].shift(i).fillna(0)
    
    for i in custom_lags:
        data['y_lag_'+str(i)] = data['volume'].shift(i).fillna(0)
      
    for col in ['CAP_BUC', 'CAT', 'holiday', 'holiday_name']:
        data[col] = data[col].astype('category')
        
    for col in ['month', 'monthday', 'year', 'week', 'weekday']:
        data[col] = data[col].astype('int')
    
    data.sort_values('date', inplace=True)
    #print(data.columns)
    return data

### Fetch data from MongoDB

In [6]:
from pymongo import MongoClient
import pandas as pd
import urllib

username = urllib.parse.quote_plus('keplerUserOnshore')
password = urllib.parse.quote_plus('Raptor2018#')
server = 'zlt21317.vci.att.com:27018' 
uri = f'mongodb://{username}:{password}@{server}'
client = MongoClient(uri,authMechanism='SCRAM-SHA-1',authSource = 'ETTR') 
db = client.ETTR 

In [7]:
# to list all the available collections :  
coll_names = db.list_collection_names()
coll_names

['AFOREPAIR_CAPBUCKET_22_Dec_2020',
 'AFOREPAIR_CAPBUCKET_09_Dec_2020',
 'REPAIR_11_Dec_2020',
 'REPAIR_03_Dec_2020',
 'AFOREPAIR_CAPBUCKET_30_Dec_2020',
 'REPAIR_16_Dec_2020',
 'REPAIR_12_Dec_2020',
 'AFOREPAIR_CAPBUCKET_28_Dec_2020',
 'AFOREPAIR_CAPBUCKET_16_Dec_2020',
 'AFOREPAIR_CAPBUCKET_06_Jan_2021',
 'test',
 'REPAIR_20_Dec_2020',
 'AFOREPAIR_CAPBUCKET_05_Dec_2020',
 'REPAIR_06_Dec_2020',
 'REPAIR_09_Jan_2021',
 'REPAIR_30_Dec_2020',
 'my_collection_keys',
 'T1682_NIFI_CPE_LOG',
 'AFOREPAIR_CAPBUCKET_15_Dec_2020',
 'BMPTKT.OPEN_MR_ACTIVITY',
 'REPAIR_03_Jan_2021',
 'AFOREPAIR_CAPBUCKET_11_Dec_2020',
 'REPAIR_22_Dec_2020',
 'AFOREPAIR_CAPBUCKET_06_Dec_2020',
 'T1682_NIFI_DIAGNOSTICS',
 'REPAIR_10_Jan_2021',
 'REPAIR_11_Jan_2021',
 'AFOREPAIR_CAPBUCKET_18_Dec_2020',
 'REPAIR_21_Dec_2020',
 'REPAIR_10_Dec_2020',
 'AFOREPAIR_CAPBUCKET_07_Jan_2021',
 'REPAIR_29_Dec_2020',
 'REPAIR_17_Dec_2020',
 'ETTR',
 'AFOREPAIR_CAPBUCKET_03_Dec_2020',
 'AFOREPAIR_CAPBUCKET_26_Dec_2020',
 'REPAIR_

In [8]:
#Best params till now

final_params = {'subsample': 0.8, 'reg_lambda': 0.4, 'reg_alpha': 0.2,
                 'objective': 'regression', 'num_leaves': 100, 'n_estimators': 500,
                 'min_child_samples': 45, 'metric': 'SMAPE', 'max_depth': 8,
                 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'boosting': 'gbdt'}

drop_columns = ['rolling_mean_21', 'rolling_mean_13', 'rolling_mean_34',
                 'rolling_std_21', 'rolling_mean_8', 'rolling_std_13', 'rolling_min_8',
                 'rolling_min_21', 'rolling_std_7', 'holiday',
                 'rolling_std_34', 'rolling_median_13']

In [10]:
train_end_date = datetime(2021, 1, 9)
pred_end_date = datetime(2021, 1, 12)

In [11]:
input_date = train_end_date
while input_date <= pred_end_date:
    start_time_date = time.time()
    pred_start_date = input_date + timedelta(days=1)
    input_collection = 'REPAIR_'+input_date.strftime('%d_%b_%Y')
    print("\n *****************************************************************************")
    print("Working on : ", input_date.date(), "\tDB name : ", input_collection)
    #break
    if input_collection not in coll_names:
        input_date = input_date + timedelta(days=1)
        continue
    actuals_df = pd.DataFrame.from_records(db[input_collection].find({}))
    actuals_df.drop('_id',axis=1,inplace=True)
    actuals_df.date = pd.to_datetime(actuals_df.date, format="%Y-%m-%d")
    print(actuals_df.shape)
    print("min :", actuals_df.date.dt.date.min())
    print("max :", actuals_df.date.dt.date.max())
    print("Total capacity bucket = {}". format(len(actuals_df['CAPACITY_BUCKET'].unique())))
    print("Total category = {}". format(len(actuals_df['CATEGORY'].unique())))

    forecast_collection = 'AFOREPAIR_CAPBUCKET_'+pred_start_date.strftime('%d_%b_%Y')
    if forecast_collection in coll_names:        
        forecast_df = pd.DataFrame.from_records(db[forecast_collection].find({}))
        forecast_df.drop('_id',axis=1,inplace=True)
        forecast_df.date = pd.to_datetime(forecast_df.date, format="%d-%b-%y")
        forecast_df.columns=['date', 'CATEGORY', 'CAPACITY_BUCKET', 'cdo_prediction']
        forecast_df[forecast_df.date == pred_start_date].to_csv("output/forecast_of_" + forecast_collection, index=False)
        print(forecast_df.shape)

        compare_df = actuals_df[['CAPACITY_BUCKET', 'CATEGORY']].drop_duplicates().merge(right=forecast_df[['CAPACITY_BUCKET', 'CATEGORY']].drop_duplicates(), indicator=True, how='left')
        print(compare_df['_merge'].value_counts())

    #Filter records to contain only capacity bucket and category which are found in last 200 days
    '''days_to_filter = 200
    filter_df = actuals_df[actuals_df.date >= datetime.today() - timedelta(days=days_to_filter)][['CAPACITY_BUCKET', 'CATEGORY']].drop_duplicates()
    filter_df['dummy'] = 1
    actuals_df = actuals_df.merge(filter_df, on=['CAPACITY_BUCKET', 'CATEGORY'], how='left')
    actuals_df = actuals_df.dropna().reset_index(drop=True)
    actuals_df.drop(columns=['dummy'], inplace=True)'''
    
    #Dummy creation and missing value fill
    actuals_df = actuals_df[actuals_df['CAPACITY_BUCKET'] != 'TX_SPORT_RA'].copy()
    actuals_df[actuals_df.date == input_date][['date', 'CAPACITY_BUCKET', 'CATEGORY', 'volume']].to_csv("output/actual_of_" + input_collection, index=False)
    #break
    print(actuals_df.shape)
    actuals_df.sort_values(['CAPACITY_BUCKET', 'CATEGORY', 'date'], inplace=True)

    # list comprehension with reindex and date_range then concat list of frames
    #actuals_df = pd.concat([df.set_index('date').reindex(pd.date_range(df['date'].min(),
    #                                                         df['date'].max(),freq='D')) for _,df in actuals_df.groupby(['CAPACITY_BUCKET', 'CATEGORY'])])
    actuals_df = pd.concat([df.set_index('date').reindex(pd.date_range(df['date'].min(),
                                                             input_date,freq='D')) for _,df in actuals_df.groupby(['CAPACITY_BUCKET', 'CATEGORY'])])
    actuals_df = actuals_df.reset_index()
    actuals_df.CAPACITY_BUCKET = actuals_df.CAPACITY_BUCKET.interpolate('pad')
    actuals_df.CATEGORY = actuals_df.CATEGORY.interpolate('pad')

    le_cap = preprocessing.LabelEncoder()
    actuals_df['CAP_BUC'] = le_cap.fit_transform(actuals_df.CAPACITY_BUCKET)

    le_cat = preprocessing.LabelEncoder()
    actuals_df['CAT'] = le_cat.fit_transform(actuals_df.CATEGORY)

    actuals_df['NAs'] = actuals_df.volume.isnull()
    actuals_df.volume.fillna(0, inplace=True)
    actuals_df.rename(columns={'index': 'date'}, inplace=True)
    
    curr_date = pred_start_date
    print("Training data available till date : ", (curr_date - timedelta(days=1)).date())
    temp_df = actuals_df[actuals_df.date <= input_date].copy()
    pred_df = []    
    while curr_date <= pred_start_date:#pred_end_date:
        start_time = time.time()
        train_end = curr_date - timedelta(days=1)    
        print("\nTraining till Date = ", train_end.date())    

        print("\t...Data Preparation started ..")
        X_test = actuals_df[['CAPACITY_BUCKET', 'CATEGORY']].drop_duplicates().reset_index(drop=True)
        X_test['CAP_BUC'] = le_cap.fit_transform(X_test.CAPACITY_BUCKET)
        X_test['CAT'] = le_cat.fit_transform(X_test.CATEGORY)
        X_test['date'] = curr_date
        X_test['volume'] = np.NaN
        X_test['NAs'] = False
        X_test = X_test[['date', 'CAPACITY_BUCKET', 'CATEGORY', 'volume', 'CAP_BUC','CAT','NAs']]

        temp_df = pd.concat([temp_df, X_test], axis=0)
        temp_df = temp_df.sort_values(['CAPACITY_BUCKET', 'CATEGORY', 'date']).reset_index(drop=True)
        #temp_df = timeseries_to_supervised(temp_df)
        mypool=Pool(processes=40)
        df_groups = temp_df.groupby(['CAPACITY_BUCKET', 'CATEGORY'])
        ret_list=mypool.imap(timeseries_to_supervised,(group for _, group in df_groups))
        temp_df = pd.concat(ret_list)
        mypool.close()

        #print("\n temp_df")
        #print(temp_df.shape)
        #print(temp_df[['date', 'volume']].tail(5))
        print("\t--- Train data preparation time : %s minutes ---" % int((time.time() - start_time) / 60))

        features = temp_df.columns.difference(['date', 'CAPACITY_BUCKET', 'CATEGORY', 'volume', 'NAs'])
        X_train = temp_df[temp_df.date <= train_end][features].copy()
        y_train = temp_df[temp_df.date <= train_end]['volume'].copy()
        #print("\n X_train")
        #print(X_train.shape)
        #print(y_train.shape)
        #print(X_train.tail())

        X_test = temp_df[temp_df.date == curr_date].copy()
        X_test = X_test[features]
        #print("\n X_test")
        #print(X_test.shape)
        #print(X_test.tail())
        #break

        print("\t...Training started ..")
        gbm = lgb.LGBMRegressor(**final_params, random_state=0, n_jobs=40)    
        gbm.fit(X_train.drop(columns=drop_columns, errors='ignore'), y_train, verbose=1)
        
        print("\t...Predicting for : ", curr_date.date())
        #print(X_test.shape)
        X_test['Pred'] = gbm.predict(X_test.drop(columns=drop_columns, errors='ignore'))
        X_test['Pred'] = np.where(X_test['Pred'] < 1, 0, X_test['Pred'])
        X_test['Pred'] = X_test['Pred'].round(2)
        X_test['date'] = curr_date
        X_test = X_test[['date', 'Pred', 'CAP_BUC', 'CAT']]
        if len(pred_df) == 0:
            pred_df = X_test.copy()
        else:
            pred_df = pd.concat([pred_df, X_test], axis=0)
        X_test['Pred'] = X_test['Pred'].round()
        temp_df = temp_df.merge(X_test, on=['date', 'CAP_BUC', 'CAT'], how='left')
        temp_df.Pred.fillna(temp_df.volume, inplace=True)
        temp_df.drop(columns=['volume'], inplace=True)
        temp_df.rename(columns={'Pred': 'volume'}, inplace=True)
        temp_df = temp_df[['date', 'CAPACITY_BUCKET', 'CATEGORY', 'volume', 'CAP_BUC','CAT','NAs']]
        #print(temp_df.volume.isnull().sum())
        curr_date = curr_date + timedelta(days=1)
        print("\t--- Total time for this iteration : %s minutes ---" % int((time.time() - start_time) / 60))
        #break

    if len(pred_df) > 0:
        pred_df['CAPACITY_BUCKET'] = le_cap.inverse_transform( pred_df.CAP_BUC )
        pred_df['CATEGORY'] = le_cat.inverse_transform( pred_df.CAT )
        pred_df.drop(columns=['CAP_BUC', 'CAT'], inplace=True)
        pred_df.rename(columns={'Pred':'prediction'}, inplace=True)
        pred_file = "output/pred_of_" + input_collection
        pred_df.to_csv(pred_file, index=False)
    
    input_date = input_date + timedelta(days=1)
    print("--- Total time for this date : %s minutes ---\n" % int((time.time() - start_time_date) / 60))
    #break


 *****************************************************************************
Working on :  2021-01-09 	DB name :  REPAIR_09_Jan_2021
(3213008, 4)
min : 2018-01-01
max : 2021-01-09
Total capacity bucket = 595
Total category = 8
(419300, 4)
both          4193
left_only      225
right_only       0
Name: _merge, dtype: int64
(3211653, 4)
Training data available till date :  2021-01-09

Training till Date =  2021-01-09
	...Data Preparation started ..
	--- Train data preparation time : 2 minutes ---
	...Training started ..
	...Predicting for :  2021-01-10
	--- Total time for this iteration : 4 minutes ---
--- Total time for this date : 5 minutes ---


 *****************************************************************************
Working on :  2021-01-10 	DB name :  REPAIR_10_Jan_2021
(3214825, 4)
min : 2018-01-01
max : 2021-01-10
Total capacity bucket = 595
Total category = 8
(419300, 4)
both          4193
left_only      225
right_only       0
Name: _merge, dtype: int64
(3213472, 4)
Train

In [12]:
train_end_date = datetime(2021, 1, 8)
pred_end_date = datetime(2021, 1, 12)

In [13]:
final_pred_df = []
input_date = train_end_date
while input_date < pred_end_date:
    print("\n *****************************************************************************")
    print("Working on : ", input_date.date())
    pred_file = "output/pred_of_" + 'REPAIR_'+input_date.strftime('%d_%b_%Y')
    print(pred_file)
    if not os.path.exists(pred_file):
        input_date = input_date + timedelta(days=1)
        continue
    pred_data = pd.read_csv(pred_file)        
    pred_data.date = pd.to_datetime(pred_data.date, format="%Y-%m-%d")
    actual_date = input_date + timedelta(days=1)
    while actual_date <= input_date + timedelta(days=1):#pred_end_date:
        actual_file = "output/actual_of_" + 'REPAIR_'+actual_date.strftime('%d_%b_%Y')
        print("...."+actual_file)
        if not os.path.exists(actual_file):
            actual_date = actual_date + timedelta(days=1)
            continue
        actuals_data = pd.read_csv(actual_file)
        actuals_data.date = pd.to_datetime(actuals_data.date, format="%Y-%m-%d")
        actuals_data = actuals_data[actuals_data.date == actual_date]
        #print(actuals_data.shape)
        #print(actuals_data.columns)
        forecast_file = "output/forecast_of_" + 'AFOREPAIR_CAPBUCKET_'+actual_date.strftime('%d_%b_%Y')
        print("...."+forecast_file)
        if os.path.exists(forecast_file):
            forecast_data = pd.read_csv(forecast_file)
            forecast_data.date = pd.to_datetime(forecast_data.date, format="%Y-%m-%d")
            forecast_data = forecast_data[forecast_data.date == actual_date]
            forecast_data = forecast_data[['CAPACITY_BUCKET', 'CATEGORY']].drop_duplicates()
            print("\t CDO prediction count : ",forecast_data.shape)
            actuals_data = actuals_data.merge(forecast_data, on=['CAPACITY_BUCKET', 'CATEGORY'], how='outer', indicator=True)         
            if actuals_data['_merge'].value_counts()['left_only'] != 0:
                print("\t Mismatch Found.")
                print(actuals_data['_merge'].value_counts())
            actuals_data['volume'].fillna(0, inplace=True)            
            actuals_data['date'] = actual_date
            actuals_data = actuals_data[['date', 'CAPACITY_BUCKET', 'CATEGORY', 'volume']]
            print("\tFinal Prediction count : ", actuals_data.shape[0])
        
            temp_df = pred_data[pred_data.date == actual_date]
            print("\t Available predictions", temp_df.shape[0])
            temp_df = temp_df.merge(actuals_data, on=['date', 'CAPACITY_BUCKET', 'CATEGORY'], how='right', indicator=True)            
            if temp_df.prediction.isnull().sum() > 0:
                print("Lesser predictions than CDO")
            temp_df.drop(columns='_merge', inplace=True)
            temp_df['Train_till_date'] = input_date
            if len(final_pred_df) == 0:
                final_pred_df = temp_df.copy()
            else:
                final_pred_df = pd.concat([final_pred_df, temp_df], axis=0)
        actual_date = actual_date + timedelta(days=1)
        #break
    input_date = input_date + timedelta(days=1)
    #break
    
final_pred_df = final_pred_df.reset_index(drop=True)
print("\nTotal null in actual", final_pred_df['volume'].isnull().sum())
print("Total null in prediction", final_pred_df['prediction'].isnull().sum())
final_pred_df['prediction'] = final_pred_df['prediction'].fillna(0)
final_pred_df.date = final_pred_df.date.dt.date
print("All predictions count : ", final_pred_df.shape[0])
final_pred_df.head()


 *****************************************************************************
Working on :  2021-01-08
output/pred_of_REPAIR_08_Jan_2021
....output/actual_of_REPAIR_09_Jan_2021
....output/forecast_of_AFOREPAIR_CAPBUCKET_09_Jan_2021
	 CDO prediction count :  (4194, 2)
	Final Prediction count :  4194
	 Available predictions 4410

 *****************************************************************************
Working on :  2021-01-09
output/pred_of_REPAIR_09_Jan_2021
....output/actual_of_REPAIR_10_Jan_2021
....output/forecast_of_AFOREPAIR_CAPBUCKET_10_Jan_2021
	 CDO prediction count :  (4193, 2)
	Final Prediction count :  4193
	 Available predictions 4410

 *****************************************************************************
Working on :  2021-01-10
output/pred_of_REPAIR_10_Jan_2021
....output/actual_of_REPAIR_11_Jan_2021
....output/forecast_of_AFOREPAIR_CAPBUCKET_11_Jan_2021
	 CDO prediction count :  (4193, 2)
	Final Prediction count :  4193
	 Available predictions 4410

 *****

date  prediction                CAPACITY_BUCKET             CATEGORY  \
0  2021-01-09       75.49        TXNO_TX_DFWT_FTW_WEST_A     RPR_IP_WORKSHARE   
1  2021-01-09       45.46  TXNO_TX_DFWT_DLS_RICHARDSON_A     RPR_IP_WORKSHARE   
2  2021-01-09        2.98        MSOUTH_IL_CGIL_JOLIET_A         RPRAS_LEGACY   
3  2021-01-09        2.60             CNC_CA_SFBY_NAPA_A  RPR_IP_NONWORKSHARE   
4  2021-01-09        4.89     FLNRT_FL_JKVL_JAXV_NORTH_A         RPRAS_LEGACY   

   volume Train_till_date  
0    80.0      2021-01-08  
1    72.0      2021-01-08  
2     2.0      2021-01-08  
3     4.0      2021-01-08  
4     4.0      2021-01-08

In [14]:
final_pred_df.loc[(final_pred_df.prediction <= 0.5) & (final_pred_df.prediction != 0)]['prediction']

Series([], Name: prediction, dtype: float64)

In [15]:
final_pred_df.tail()

date  prediction                     CAPACITY_BUCKET  \
12575  2021-01-11         0.0                 PNW_CA_CHCO_CHICO_A   
12576  2021-01-11         0.0                  CNC_CA_SFBY_LAKE_A   
12577  2021-01-11         0.0     TXNO_TX_WACO_FTW_WCO_HLMGRT_2_A   
12578  2021-01-11         0.0  KYTEN_MS_MEMT_MEMPHIS_WEST_METRO_A   
12579  2021-01-11         0.0            SNV_CA_SACR_S_TAHOE_TU_A   

          CATEGORY  volume Train_till_date  
12575  RPRIP_DDCIM     0.0      2021-01-10  
12576  RPRIP_DDCIM     0.0      2021-01-10  
12577  RPRIP_DDCIM     0.0      2021-01-10  
12578    RPR_CABLE     0.0      2021-01-10  
12579    RPR_CABLE     0.0      2021-01-10

In [16]:
final_pred_df.groupby(['Train_till_date', 'date'])[['prediction']].sum().reset_index().pivot(index='Train_till_date', columns='date', values='prediction').reset_index().fillna('')

date Train_till_date 2021-01-09 2021-01-10 2021-01-11
0         2021-01-08    15818.9                      
1         2021-01-09               8639.56           
2         2021-01-10                          28909.9

In [17]:
smape_df = final_pred_df.groupby(['Train_till_date', 'date'])[['volume', 'prediction']].apply(lambda x: smape(x.volume, x.prediction)).reset_index()
smape_df.columns = ['Train_till_date', 'date', 'smape']
smape_df = smape_df.pivot(index='Train_till_date', columns='date', values='smape').reset_index()
smape_df.fillna('')

date Train_till_date 2021-01-09 2021-01-10 2021-01-11
0         2021-01-08     0.1803                      
1         2021-01-09                0.2166           
2         2021-01-10                           0.1637